<a href="https://colab.research.google.com/github/mhrgroup/course_self_supervised_learning/blob/main/Section%2002%3A%20Supervised%20Models/ssl_section02_lecture04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lecture 04: Transfer Learning & Fine-Tuning**

By the end of this lecture, you will be able to:

1. Implement supervised learning with transfer learning and fine-tuning.



# **4.1. Supervised Learning with Transfer Learning and Fine-Tuning**
---

* One can develop a supervised model using an already trained model.

* An already trained model, referred to as the base model, is usually borrowed entirely or partially and modified to learn the current problem quickly using transfer learning and fine-tuning techniques.

> <img src=	"	https://raw.githubusercontent.com/mhrgroup/course_self_supervised_learning/main/images/basemodel.png	"	width="150"/>

* A base model usually includes hundreds of convolutional, recurrent, or dense (i.e., fully connected) layers. These layers have been trained (for days) on a different data set than the current problem using expensive resources. Such datasets might have thousands of categories or include millions of data points.

* The base model is enriched with learned parameters, weights, filters, and kernels capable of identifying feature representation for any problem within the same data domain as the one base model was trained on.

* The base model’s input and output layers are often modified to be compatible with the current problem. For example, non-trainable lower/upper scaling layers are added to the input layer for input dimension compatibility. Output layer/s are substituted with trainable dense/SoftMax layers compatible with the current dataset’s output.

> <img src=	"	https://raw.githubusercontent.com/mhrgroup/course_self_supervised_learning/main/images/modification.png	"	width="175"/>

* The base model (or a portion of the base model) is frozen in transfer learning, meaning its parameters are not updated. Instead, the newly added output layer is trained using a relatively large learning rate for many epochs.

> <img src=	"	https://raw.githubusercontent.com/mhrgroup/course_self_supervised_learning/main/images/transferlearning.png	"	width="175"/>

* The base model already produced enriched feature representation from data; the only task is to learn from this representation to address the current problem using the newly added output layer.

* The base model (or a portion of the base model) is unfrozen in fine-tuning. Its parameters can now be updated using a relatively small learning rate for a few epochs to prevent overfitting, especially if the current dataset has few labeled data. Fine-tuning often boosts the accuracy of the model significantly.

> <img src=	"	https://raw.githubusercontent.com/mhrgroup/course_self_supervised_learning/main/images/finetuning.png	"	width="175"/>

* The whole process is shown in this figure:

> <img src=	"	https://raw.githubusercontent.com/mhrgroup/course_self_supervised_learning/main/images/ssl0.png	"	width="650"/>

* Let's practice this using the CIFAR-10 dataset

> **Abbreviations:**
*	datain: input data
*	dataou: output data
*	te: testing
*	tf: tensorflow
*	tr: training

In [ ]:
#@title Install cecessary libraries & restart the session

# Install the required libraries using the `pip` package manager.
!pip install tensorflow==2.15

# Import the time module to add a delay before restarting the session.
import time

# Import `clear_output` from IPython to clear the notebook output, ensuring a clean display for the user.
from IPython.display import clear_output

# Clear the output after the packages are installed to make the notebook cleaner.
clear_output()

# Print a message to let the user know that the libraries are installed & the session will restart.
print("Necessary Libraries are Installed. Restarting the session!")

# Add a short delay (1 second) before restarting to allow the message to be displayed to the user.
time.sleep(1)

# Import the `os` module to access low-level operating system functionality.
import os

# Use `os._exit(00)` to exit the current Python runtime environment forcefully.
# This effectively simulates a restart in notebook environments like Google Colab or Jupyter.
# After this command, the environment will be restarted & all the packages installed will be properly loaded.
os._exit(00)

In [ ]:
#@title Import necessary libraries
import time
import tensorflow as tf

In [ ]:
#@title Load and process the CIFAR-10 data
(datain_tr, dataou_tr), (datain_te, dataou_te) = tf.keras.datasets.cifar10.load_data()

'''
List of available datasets at tf.keras.datasets:
https://www.tensorflow.org/api_docs/python/tf/keras/datasets.
'''

datain_tr = datain_tr/255 # trasnform unit-8 values between 0 and 1
datain_te = datain_te/255 # trasnform unit-8 values between 0 and 1

dataou_tr = tf.keras.utils.to_categorical(dataou_tr)
dataou_te = tf.keras.utils.to_categorical(dataou_te)

# print shapes of data

print('Shape of datain_tr: {}'.format(datain_tr.shape))
print('Shape of datain_te: {}'.format(datain_te.shape))
print('Shape of dataou_tr: {}'.format(dataou_tr.shape))
print('Shape of dataou_te: {}'.format(dataou_te.shape))


In [ ]:
#@title Create a model similar to DenseNet121
'''
DenseNet121 is a deep neural network tower including convolutional and dense
layers. You can learn about it at
https://arxiv.org/abs/1608.06993
'''

layerin = tf.keras.Input(shape=(datain_tr.shape[1],
                                datain_tr.shape[2],
                                datain_tr.shape[3]))

'''
The DenseNet121 input size is 160 by 160 by 3, and our images are 32 by 32 by 3.
The upscale is to resize our 32 by 32 images to become 160 by 160 for
compatibility

Read about tf.keras.layers.Lambda here:
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Lambda
'''

upscale = tf.keras.layers.Lambda(lambda x: tf.image.resize_with_pad(x,
                                                                    160,
                                                                    160,
                                                                    method=tf.image.ResizeMethod.BILINEAR))(layerin)
'''
Read more about image ResizeMethod at
https://www.tensorflow.org/api_docs/python/tf/image/resize
'''


# Here we are going to use ImageNet weights to create our base model
model_base = tf.keras.applications.DenseNet121(include_top=False,
                                              weights = 'imagenet',
                                              input_shape  = (160,160,3),
                                              input_tensor = upscale,
                                              pooling='max')

'''
read more at
https://www.tensorflow.org/api_docs/python/tf/keras/applications/densenet/DenseNet121
'''

layerou = tf.keras.layers.Dense(dataou_tr.shape[-1], activation = 'softmax')

'''
Create the model!
We add "BatchNormalization" to reduce overfitting.
A useful article to learn more about BacthNormalization:
https://towardsdatascience.com/batch-norm-explained-visually-how-it-works-and-why-neural-networks-need-it-b18919692739
'''

model   = tf.keras.models.Sequential([model_base,
                                      tf.keras.layers.BatchNormalization(),
                                      layerou])

model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss      = 'categorical_crossentropy',
              metrics   = ['accuracy'])


'''
Print model architectures
'''

print('summary of model_DenseNet121:\n')
model_base.summary()

print('summary of model:\n')
model.summary()

In [ ]:
#@title Transfer learning
# Freeze base model and train the last layer only
model_base.trainable = False

'''
When you freeze model_base, it is also being frozen within the model; they are
all at the exact location on your memory, but you have to compile the model
again.
'''

model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
              loss      = 'categorical_crossentropy',
              metrics   = ['accuracy'])

print('\nsummary of model_DenseNet121:\n')
model_base.summary()

print('\nsummary of model:\n')
model.summary()

'''
Train for three epochs
'''

t_start = time.time()

history = model.fit(datain_tr, dataou_tr, epochs = 3, batch_size = 128,
                    verbose = 1, shuffle = True, validation_split = 0.05)

t_end   = time.time()

t_transfer_learning = t_end - t_start

print('\nTransfer learning training time: {:06.2f} sec'.format(t_transfer_learning))


In [ ]:
#@title Fine-tuning

# Unfreez the base model and train the last layer only
model_base.trainable = True

'''
When you unfreeze model_base, it is also being unfrozen within the model;
they are all at the exact location on your memory, but you have to compile the
model again.
'''

model.compile(optimizer = tf.keras.optimizers.Adam(0.00001),
              loss      = 'categorical_crossentropy',
              metrics   = ['accuracy'])

print('summary of model_DenseNet121:\n')
model_base.summary()

print('summary of model:\n')
model.summary()

'''
Train for two epochs
'''

t_start = time.time()

history = model.fit(datain_tr,
                    dataou_tr,
                    epochs           = 2,
                    batch_size       = 128,
                    verbose          = 1,
                    shuffle          = True,
                    validation_split = 0.05)

t_end   = time.time()

t_fine_tuning = t_end - t_start

print('\nFine-tuning training time: {:06.2f} sec'.format(t_fine_tuning))


In [ ]:
#@title Compute testing accuracy
_, accuracy_te = model.evaluate(datain_te,
                                dataou_te,
                                batch_size = 128)

print('\nTotal Training time: {:06.2f} sec'.format(t_transfer_learning + t_fine_tuning))
print('\nTesting acuuracy: {:05.2f}%'.format(accuracy_te * 100))

In [ ]:
#@title Clean up memory
%reset

# **Lecture 04: Transfer Learning and Fine-Tuning**

In this lecture, you learned about:

1. supervised learning with transfer learning and fine-tuning.

> ***In the following lecture, we will discuss "Labeling Challenges."***